<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/scene-index/integrations/llama-index/simple_video_rag_with_readers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# VideoDB Readers 

### Note : Draft Notebook & Integrations


&nbsp;
## 🛠️️ Setup connection

###  Requirements

To connect to VideoDB, simply get the API key and create a connection. This can be done by setting the `VIDEO_DB_API_KEY` environment variable. You can get it from 👉🏼 [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required!** )

Get your `OPENAI_API_KEY` from OpenAI platform for `llama_index` response synthesizer.

<!-- > Set the `OPENAI_API_KEY` & `VIDEO_DB_API_KEY` environment variable with your API keys. -->

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["VIDEO_DB_API_KEY"] = ""

### Installing Dependencies

To get started, we'll need to install the following packages:

- `llama-index`
- `llama-index-readers-videodb`
- `videodb`

In [ ]:
%pip install llama-index
%pip install videodb

In [ ]:
%pip install git+https://github.com/video-db/llama_index@add-videodb-readers#subdirectory=llama-index-integrations/readers/llama-index-readers-videodb

### Data Ingestion

Let's upload a few video files first. You can use any `public url`, `Youtube link` or `local file` on your system. First 50 uploads are free!

In [ ]:
from videodb import connect

# connect to VideoDB
conn = connect()

# upload videos to default collection in VideoDB
print("uploading first video")
video = conn.upload(url="https://www.youtube.com/watch?v=zdPSaMuLRso&list=PL6duHAYny9nNuTHN2EKNfuhmmRKCJD7os&index=5")

> * `coll = conn.get_collection()` : Returns default collection object.
> * `coll.get_videos()` : Returns list of all the videos in a collections.
> * `coll.get_video(video_id)`: Returns Video object from given`video_id`.

### Index Spoken Words 

In [ ]:
video.index_spoken_words()

### Indexing scenes


In [ ]:
from videodb import SceneExtractionType

index_id = video.index_scenes(extraction_type=SceneExtractionType.time_based, extraction_config={"time": 30}, model_name="gemini-1.5-pro")
scenes = video.get_scene_index(index_id)

#NOTE: replace this by your scn collection id
scn_col_id = "tt30sf"

### SimpleRAG

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.videodb import VideoDBReader

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

reader = VideoDBReader(video=video.id, base_url="https://api.videodb.io")

scene_nodes = reader.get_scene_nodes(scn_col_id=scn_col_id)
transcript_nodes = reader.get_transcript_nodes(50)

index = VectorStoreIndex(scene_nodes+transcript_nodes)

query_engine = index.as_query_engine()

In [ ]:
res = query_engine.query("Projection of a vector")
print(res)

In [ ]:
for node in res.source_nodes:
    print(node.metadata)
    print(node.text)

In [ ]:
from videodb import connect, play_stream
from videodb.timeline import Timeline
from videodb.asset import VideoAsset

def preview_nodes(nodes, video_id):
    # TODO: Support cross collection merge
    sorted_shots= sorted([(node.metadata['start'], node.metadata['end']) for node in nodes], key=lambda x: x[0])

    merged_shots = []
    for start, end in sorted_shots:
        if not merged_shots or start > merged_shots[-1][1]:
            merged_shots.append((start, end))
        else:
            merged_shots[-1] = (merged_shots[-1][0], max(merged_shots[-1][1], end))

    conn = connect(base_url="https://api.dev.videodb.io")
    timeline = Timeline(conn)

    for shot in merged_shots:
        video_asset = VideoAsset(asset_id=video_id, start=shot[0], end=shot[1])
        print("adding video_asset ", video_asset)
        timeline.add_inline(video_asset)
    
    stream_url = timeline.generate_stream()
    return play_stream(stream_url)

print(preview_nodes(res.source_nodes, video.id))